02で前処理をしたデータの読み込みと03で学習したモデルを使って推論を行うためのnotebookです。  

## 必要なライブラリのimport

In [6]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pickle

In [7]:
# 予測したいデータの読み込み
test = pd.read_csv('../data/processed/processed20240627_test.csv')

## 保存されたモデルを使って予測を行う

In [5]:
n_folds = 5
predictions = np.zeros(len(test))
features = [c for c in test.columns if c not in ['card_id', 'first_active_month']]

for fold_ in range(n_folds):
    # モデルを読み込む
    with open(f'../src/models/model_fold_{fold_}.pkl', 'rb') as f:
        clf = pickle.load(f)
    
    # モデルで予測を行う
    fold_predictions = clf.predict(test[features])
    
    # 予測結果を足し合わせる
    predictions += fold_predictions

# 平均を取る
predictions /= n_folds

# RMSEを計算する（テストデータに対して真の値がある場合）
# y_testが存在する場合に限り
if 'y_test' in locals():
    score = mean_squared_error(y_test, predictions, squared=False)
    print(f'RMSE={score}')

TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'pandas.core.frame.DataFrame'>)

In [ ]:
# 予測結果を出力する
print(predictions)

[-4.11878134 -0.35235495 -1.43520575 ...  0.88413098 -4.80468392
  0.15235199]


## 提出用ファイルの作成

In [ ]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submit.csv", index=False)